In [1]:
from keystone.model import Net
from main import train
from types import SimpleNamespace

In [2]:
training_params = SimpleNamespace(
        file_path = './data/Ptrain.csv',
        batch_size = 32,
        test_size = 0.0,
        lr = 0.001,
        epochs = 100,
        verbosity = 10,
        layers = [1000]
    )

In [3]:
train(training_params)

Epoch: 0 	Training Loss: 6.2855405e-05 	Validation Loss: nan
Epoch: 50 	Training Loss: 2.3429898e-06 	Validation Loss: nan
Epoch: 100 	Training Loss: 1.2819780e-06 	Validation Loss: nan


KeyboardInterrupt: 